<p align="center">
<img src="https://i.imgur.com/XaHgL55.png">
</p>




# About the competition:
<img align="right" width="250" src="https://storage.googleapis.com/kaggle-media/competitions/Sartorius/Sartorius_Competition%20Description%20Image%20350x379.png" />

Howdy! It has been 20 whole days and the competition has started picking up speed. The leaderboard is changing every hour. I always wanted to learn instance segmentation and I got really excited when I saw a Kaggle competition on it `ToT` . Apparently every year kaggle hosts one or more instance segmentation competitions, like this year's [iMaterialist (Fashion) 2021 at FGVC8](https://www.kaggle.com/c/imaterialist-fashion-2021-fgvc8) competition. Although It was not much popular. The in-hand task for this competition is to detect the boundaries of each cell body using segmentation. In this competition we are segmenting neuronal cells in images.The training annotations are provided as run length encoded masks, and the images are in PNG format. The number of images is small, but the number of annotated objects is quite high.The hidden test set is roughly 240 images. Annotations are a bit noisy in some places because of the small size of cell bodies.




> Yeah you know that. We just need to do segmentation for this competition to detect the boundaries of a neuron cell. But how does this help the medical industry? Actually different neurological disorders, including neurodegenerative diseases such as Alzheimer's and brain tumors are a leading cause of death and disability across the globe. And (I guess) these diseases affect the brain cells, mainly the cell body. So for detecting which part of the brain has problems in the cell body we need the help of computer vision. And with the help of segmented output for the region of interest researchers and doctors will be able to analyze properly and help the patient.




## Why we are using Unet when it is a instance segmentation Competition:

<img align="center" src="https://i.imgflip.com/5t9p8v.jpg" title="made at imgflip.com"/>

> If you are getting started with the competition then, there might be Question like, **"This is a instance segmentation competition then why there are so many Notenooks on Unet which is a sementic segmentation model?"** So, let me explain, we are actually doing Instance segmentation but not directly, It is a post processing part now. If we apply instance segmentation in this daataset we will get the mask for each seperated cell instances with their class labels and bounding boxes, As we dont need class informatiopn and bounding box for the submission thats why we fall back to sementic segmentation and we seperate the segmented cells from a predicted mask using opencv **`connectedComponents`** is. You can see the post-processing part of this NB  [🦠 Sartorius - Starter Baseline Torch U-net](https://www.kaggle.com/julian3833/sartorius-starter-baseline-torch-u-net#Predict)  by [@julian3833](https://www.kaggle.com/julian3833) in the Predict/Utilities part. I also took that code for preprocessing for my NB.

> > Check out more about Unet utilization, here
> > - [UNet Strikes Back 🔥 | LB: 0.155+](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/286553)
> > - [Tips in submission and baseline (in the beginning of the competition)](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/279790)
> > - [Train Attention based Residual Unet with watershed algorithm for instance segmentation](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/285422) [mainly check out the comments ]
> > - [Are we predicting cells or a mask?](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/287487)[mainly check out the comments ]

## About this NB:

> 1. In this notebook I have tried to do a thorough analysis of different cell types given in the dataset.
> 2. Did some visualizations for different types of augmentations.
> 3. And mainly I tried to Implement Attention based Residual Unet. It is a scratch implementation.
> 4. Used tf.data for data pipeline.
> 5. **Tried to make this end to end, from EDA to submission.**


## Let you get started with the competition:
> If you are relatively new to Computer Vision, then starting out with Segmentation might be a bit hectic. But I have tried to explain everything thoroughly  with images. most of the code is with comments. I have been following the notebook section as well as the discussion section and I have some personal favorites that you must check out if you are late in the competition.

> > #### Notebooks:
> > 1. [🦠 Sartorius - Classifier + Mask R-CNN [LB=0.28]](https://www.kaggle.com/julian3833/sartorius-classifier-mask-r-cnn-lb-0-28)
> > 2. [Sartorius Cell Instance Segmentation - EDA](https://www.kaggle.com/gunesevitan/sartorius-cell-instance-segmentation-eda)
> > 3. [Positive score with Detectron 3/3 - Inference](https://www.kaggle.com/slawekbiel/positive-score-with-detectron-3-3-inference)
> > 4. [Sartorius Segmentation - Keras U-Net [Training]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-training)


> > #### Discussions:
> > 1. [Tips in submission and baseline (in the beginning of the competition)](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/279790)

> > 2. [Annotations Are too Noisy for the Metric](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/281205)

> > 3. [[Info] Instance Segmentation Models (quick list)](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/278883)

> > 4. [tutorial detectron 2](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/280137)

> > 5. [Overlaps-ambiguous or not?](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/280250)

> > 6. [No Overlap Issue: Quick Fix](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/279995)


<font size="4"
          face="verdana"
          color="red">
            If you find this kernel useful, Please Upvote. Thank you.  
        </font>

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from plotly.offline import iplot
import plotly as py
import plotly.tools as tls
import cufflinks as cf
from IPython.core.display import display, HTML
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
import keras
import imgaug.augmenters as iaa
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K

# defining the losses
# from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from datetime import datetime 
from PIL import Image
import albumentations as A

# from keras import backend, optimizers
#

AUTOTUNE = tf.data.experimental.AUTOTUNE
plt.style.use('fivethirtyeight')
py.offline.init_notebook_mode(connected = True)
cf.go_offline()
%matplotlib inline

In [ ]:
# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [ ]:
# import wandb
# from wandb.keras import WandbCallback

# wandb.login()

In [ ]:
import wandb
from wandb.keras import WandbCallback

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
# !pip install chart-studio -q
# %%capture
!pip install ../input/detoxify-wheel/segmentation_models-1.0.1-py3-none-any.whl

# Data Analysis and Preprocessing

In [ ]:
train_img = "../input/sartorius-cell-instance-segmentation/train"
train_csv = "../input/sartorius-cell-instance-segmentation/train.csv"

df = pd.read_csv(train_csv)
df.head()

In [ ]:
len(np.unique(df["id"])) #number of unique ids

In [ ]:
len(df["id"]) #total no of records present

In [ ]:
df["cell_type"].value_counts().iplot(kind='bar',color='blue') #different types of cells in the dataset

> This shows the number of annotated cells are present in the `shsy5y` type cell. And the count ofr `cort` and `astro` looks similar.

In [ ]:
plt.figure(figsize=(15,5))
plt.bar([0,1,2],[len(np.unique(df[df['cell_type']=='shsy5y']['id'])),len(np.unique(df[df['cell_type']=='cort']['id'])),len(np.unique(df[df['cell_type']=='astro']['id']))], label="Data 1")
plt.legend()

plt.xlabel('shsy5y, cort and astro respectively')
plt.ylabel('unique count')
plt.title('cell type bar chart')

plt.show()

> It shows the count of the cell types are `cort>shsy5y>astro`

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)
    



def build_masks(image_id,input_shape, colors=True):
    height, width = input_shape
    labels = df[df["id"] == image_id]["annotation"].tolist()
    if colors:
        mask = np.zeros((height, width, 3))
        for label in labels:
            mask += rle_decode(label, shape=(height,width , 3), color=np.random.rand(3))
    else:
        mask = np.zeros((height, width, 1))
        for label in labels:
            mask += rle_decode(label, shape=(height, width, 1))
    mask = mask.clip(0, 1)
    return mask
    

In [ ]:
ide = 'ffdb3cc02eef'
sample_image_df = df[df['id'] == ide]
sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
sample_img = cv2.imread(sample_path)
print(sample_img.shape)
sample_masks1=build_masks(ide,input_shape=(520, 704), colors=False)
sample_masks1.shape

In [ ]:
ids = ['fe3e30f849f4','ffdb3cc02eef','0140b3c8f445']
COLOR_MAP = 'seismic'


def load_data_viz(ids):
    loaded_data = []
    for ide in ids:
        sample_image_df = df[df['id'] == ide]
        sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
        sample_img = plt.imread(sample_path)
        sample_masks1=build_masks(ide,input_shape=(520, 704), colors=False)
        sample_masks2=build_masks(ide,input_shape=(520, 704), colors=True)
        loaded_data.append([sample_img,sample_masks1,sample_masks2])
    return loaded_data

data = load_data_viz(ids)

    
plt.figure(figsize=(14,14))
plt.suptitle("subplots of shsy5y,cort and astro",fontweight="bold", size=20)

plt.subplot(3, 3, 1)
plt.imshow(data[0][0],cmap = 'seismic')
plt.title("shsy5y iamge")


plt.subplot(3, 3, 2)
plt.imshow(data[0][1],cmap='gray')
plt.title("shsy5y mask iamge")

plt.subplot(3, 3, 3)
plt.imshow(data[0][2])
plt.title("shsy5y color mask iamge")



plt.subplot(3, 3, 4)
plt.imshow(data[1][0],cmap = 'seismic')
plt.title("cort iamge")

plt.subplot(3, 3, 5)
plt.imshow(data[1][1],cmap='gray')
plt.title("cort mask iamge")

plt.subplot(3, 3, 6)
plt.imshow(data[1][2])
plt.title("cort color mask iamge")



plt.subplot(3, 3, 7)
plt.imshow(data[2][0],cmap = 'seismic')
plt.title("astro iamge")

plt.subplot(3, 3, 8)
plt.imshow(data[2][1],cmap='gray')
plt.title("astro mask iamge")

plt.subplot(3, 3, 9)
plt.imshow(data[2][2])
plt.title("astro color mask iamge")



> Structure wise `shsy5y` cells look bigger than the `cort` types but smaller than the `astro` types.

# Analysis on different cell images provided in the dataset:
## shsy5y
Lets talk about the shsy5y type cells, 
SH-SY5Y is a human derived cell line used in scientific research. And it has been used to study Parkinson's disease, neurogenesis, and other characteristics of brain cells. The main structure that is mainly annotated is the SOMA(cell body). neuclear is situated over here. Axon and dendron parts are not included in the annotations. Its not only for the sh-sy5y type cells, its for other two types also(not sure about the `astro` cell type though). `shsy5y` comes under the `neuroblastoma` cell line.
<!-- ![]() -->
<p align="center">
<img width = "300" src="https://www.researchgate.net/profile/Dianne-Langford/publication/256102174/figure/fig2/AS:601664889319463@1520459496437/Differentiated-SH-SY5Y-cells-Cells-do-not-cluster-and-have-a-more-pyramidal-shaped-cell.png">
</p>

<!-- I used to bunl mu biology classes I did not know it will take revenge like this. -->

In [ ]:
ids = ['fe3e30f849f4','ffdb3cc02eef','0140b3c8f445']
COLOR_MAP = 'seismic'


def load_data_viz(ids):
    loaded_data = []
    for ide in ids:
        sample_image_df = df[df['id'] == ide]
        sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
        sample_img = cv2.imread(sample_path,cv2.IMREAD_COLOR)
        loaded_data.append(sample_img)
    return loaded_data



aug_data = []
def get_aug(sample_img):
    aug1 = iaa.AllChannelsCLAHE(clip_limit=(1, 10), per_channel=True)(image=sample_img)
    aug2 = iaa.LogContrast(gain=(0.6, 1.4), per_channel=True)(image=sample_img)
    aug3 = iaa.BlendAlpha((0.0, 1.0), iaa.HistogramEqualization())(image=sample_img)
    aug4 = iaa.Canny(alpha=(0.1, 0.8))(image=sample_img)
    aug5 = iaa.Canny(alpha=(0.0, 0.5),colorizer=iaa.RandomColorsBinaryImageColorizer(color_true=255,color_false=0))(image=sample_img)
    aug6 = iaa.Canny(alpha=(0.4, 0.5), sobel_kernel_size=[3, 7])(image=sample_img)
    aug_data.append([aug1,aug2,aug3,aug4,aug5,aug6])
    return aug_data


cell_img = load_data_viz(ids)
aug_img1 =  get_aug(cell_img[0])
aug_img2 =  get_aug(cell_img[1])
aug_img3 =  get_aug(cell_img[2])

In [ ]:
def get_img_paths(cell_type, i,j): 
    ids = np.unique(df[df['cell_type']==cell_type]['id'])[i:j]
    paths = []
    for ide in ids:
        sample_image_df = df[df['id'] == ide]
        sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
        paths.append(sample_path)
    return paths


def show_img_subplot(i,j,title_text,cell_type = 'shsy5y'):
    images_paths = get_img_paths(cell_type,i,j)
    plt.style.use('fivethirtyeight')
    figure, ax = plt.subplots(nrows=4,ncols=4,figsize=(16,16) )
    plt.suptitle(title_text,fontweight="bold", size=20)
    for ind,image_path in enumerate(images_paths):
        image=plt.imread(image_path)
        try:
            ax.ravel()[ind].imshow(image,cmap = 'seismic')
            ax.ravel()[ind].set_axis_off()
        except:
            continue;
    plt.tight_layout()
    plt.show()
    
show_img_subplot(23,39,"All shsy5y type cells",'shsy5y')

In [ ]:
method_names = ["AllChannelsCLAHE","LogContrast","BlendAlpha","Canny","RandomColorsBinaryImageColorizer","canny with soble kernel"]

def plot(data,data_idx = 0,img_idx = 0,text="shsy5y images"):
    display(HTML('<h1 style="background-color: #2dfafa;align="center">Original Image Vs Different Augmentations</h1>'))
    for i,j in enumerate(range(1,13,2)):
        plt.figure(figsize=(30,30))
        plt.subplot(6, 2, j)
        plt.title(text)
        plt.imshow(cell_img[img_idx])
        plt.subplot(6, 2, j+1)
        plt.imshow(data[data_idx][i])
        plt.title(method_names[i])
        plt.tight_layout()
        plt.show()


plot(aug_img1,0,0,text = "shsy5y images")

## Lets talk about `cort` cells:
I was reading the competition overview, but there is nothing about the cell type named `cort`, and apperantly there is nothing much about this in the internet too. But by the picture in the overview section I assume that the `neurons` are the `cort`. And I have reasons to belive that, see the 2nd bar chart from the top of the NB. actually thats the bar plot for the count of brain cell for the data provided, and you can see that the `cort` type clearly winning the race and it is because in general the number of `neurons` or `cort` are in majority in the brain of a normal human being. So, we can say that collecting data for `cort` is more easier that other ones.

> According to the visualizations we can say that the smallest cell in the dataset is `cort` and according to the google search, the cell body of a motor neuron is approximately 100 microns (0.1 millimeter) in diameter and I believe its a smaller size compared to the others. And this might lead to missing of some of the cells  from the segmentation output. I have seen 1 or 2 discussion threads regarding this topic that the [`Annotations Are too Noisy`](https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/281205), and the main reason behind that is the cells are too small and because of that annotations are not much clear. [@theoviel](https://www.kaggle.com/theoviel) explained that very nicely, go check that out.

In [ ]:
show_img_subplot(23,39,"All cort type cells",'cort')

## Data Augmentation for `cort`:
I have not done much analysis on augmentations though. But watching these visualizations made me think that, as the cells are not much big so sticking with the augmentations that highlights the cells very brightly/ boldly would be a good idea, and of course with all the other rotation and flipping and random cropping type augs. I have tried mainly six augmentations, which are mainly `AllChannelsCLAHE,LogContrast,BlendAlpha,Canny, canny with RandomColorBinaryImageColorizer and canny with soble kernel`. Using blur, [meta](https://imgaug.readthedocs.io/en/latest/source/overview/meta.html), [segmentation](https://imgaug.readthedocs.io/en/latest/source/overview/segmentation.html) or gaussian blur/ noise type of augmentation might not be that helpful.



In [ ]:
plot(aug_img2,1,1)

# The last one standing:
`The most common brain cells are neurons and non-neuron cells called glia.` Astrocytes, the most abundant glial cell type in the brain,And for a fact these cell types are well known to provide metabolic and trophic support to neurons and modulate synaptic activity. I bet you did not know that probably dont care about that either, like me `XD`. Anyways looking at the visualizations it feels like these are the cells which are bigger in size and probably annotated more precisely.  


<!-- might sound like im a doctor, but... -->

In [ ]:
show_img_subplot(23,39,"All astrocyte type cells",'astro')

### Below are the augmentations visualizations for the astrocyte cells.  

In [ ]:
plot(aug_img3,2,2)

In [ ]:
import imgaug.augmenters as iaa
ide = "fe3e30f849f4"
sample_image_df = df[df['id'] == ide]
sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
sample_img = cv2.imread(sample_path,cv2.IMREAD_COLOR)
# sample_img = plt.imread(sample_path)
# sample_img = sample_img.astype('float16')
aug = iaa.BlendAlpha(
    (0.0, 1.0),
    iaa.Affine(rotate=(-20, 20)),
    per_channel=0.5)(image=sample_img)

plt.figure(figsize=(14,14))
plt.subplot(1,2,1)
plt.imshow(sample_img,cmap="seismic")
plt.title("normal")
plt.subplot(1,2,2)
plt.imshow(aug,cmap="seismic")
plt.title("BlendAlpha")

## Data pipeline:
The load_data function loads the dataset and returns the training, validation and testing dataset. It takes the dataset path and the dataset split value as the argument. I gives the list of directories for train and test. After that we will be using `tf.data` for loading the data on the fly.

In [ ]:
def load_data(mask_path, split=0.1):
    img_path = "../input/sartorius-cell-instance-segmentation/train"
    images = [img_path + "/"+ i for i in sorted(os.listdir("../input/sartorious-nb-1-data-preprocessing-visualization/masks/img"))]
    masks = [mask_path + "/"+ i for i in sorted(os.listdir("../input/sartorious-nb-1-data-preprocessing-visualization/masks/img"))]

    train_x, valid_x = train_test_split(images, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split, random_state=42)


    return (train_x, train_y), (valid_x, valid_y)

In [ ]:
mask_path = "../input/sartorious-nb-1-data-preprocessing-visualization/masks/img"
(train_x, train_y), (valid_x, valid_y) = load_data(mask_path)

In [ ]:
train_x = np.array(train_x)
train_y = np.array(train_y)
valid_x = np.array(valid_x)
valid_y = np.array(valid_y)

In [ ]:
def load_image_and_label_from_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
#     print(f"image: {img}")

    img = tf.image.decode_png(img, channels=3)
    
    mask = tf.io.read_file(mask_path)
#     print(f"mask: {mask}")
    mask = tf.image.decode_png(mask, channels=1)
#     print(image_path)
#     img = read_image(image_path)
#     mask = read_mask(mask_path)
    
    
    
    return img, mask

In [ ]:
training_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))

# training_data = tf.data.Dataset.from_tensor_slices((df["img_path"].values, df["mask_path"].values))
# validation_data = tf.data.Dataset.from_tensor_slices((val_data["filepath"].values, val_data["label"].values))
validation_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
validation_data = validation_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
# for i in training_data:
#     print(i[0].numpy().decode("utf-8") )
#     break


In [ ]:
def augment_train_data(train_ds):
    transforms = A.Compose([
            A.RandomResizedCrop(image_size, image_size),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
            ], p=1)
    
    def aug_fn(image,mask):
#         data_img = {"image":image}
#         aug_data_img = transforms(**data_img)
        
#         data_mask = {'mask':mask}
#         aug_data_mask = transforms(**data_mask)

        aug_data_all = transforms(image=image,mask=mask)
        aug_img = aug_data_all["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        
        aug_mask = aug_data_all["mask"]
        aug_mask = tf.cast(aug_mask, tf.float32)
        return aug_img,aug_mask

    def process_data(image, mask):
        aug_img,aug_mask = tf.numpy_function(func=aug_fn, inp=[image,mask], Tout=[tf.float32,tf.float32])
        return aug_img, aug_mask
    
    def set_shapes(img, mask, img_shape=(image_size,image_size,3)):
        img.set_shape(img_shape)
        mask.set_shape([img_shape[0],img_shape[1],1])
        return img, mask
    
    ds_alb = train_ds.map(partial(process_data), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=AUTOTUNE)
    ds_alb = ds_alb.repeat()
    ds_alb = ds_alb.batch(batch)
    return ds_alb

In [ ]:
def augment_val_data(val_ds):
    transforms = A.Compose([
                A.CenterCrop(image_size, image_size),
                ], p=1)
#     transforms = A.Compose([
#             A.RandomResizedCrop(image_size, image_size),
#             A.Transpose(p=0.5),
#             A.HorizontalFlip(p=0.5),
#             A.VerticalFlip(p=0.5),
#             A.ShiftScaleRotate(p=0.5),
#             A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#             A.CoarseDropout(p=0.5),
#             A.Cutout(p=0.5),
#             ], p=1)
    
    def aug_fn(image,mask):
#         data_img = {"image":image}
#         aug_data_img = transforms(**data_img)
        
#         data_mask = {'mask':mask}
#         aug_data_mask = transforms(**data_mask)

        aug_data_all = transforms(image=image,mask=mask)
        aug_img = aug_data_all["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        
        aug_mask = aug_data_all["mask"]
        aug_mask = tf.cast(aug_mask, tf.float32)
        return aug_img,aug_mask

    def process_data(image, mask):
        aug_img,aug_mask = tf.numpy_function(func=aug_fn, inp=[image,mask], Tout=[tf.float32,tf.float32])
        return aug_img, aug_mask
    
    def set_shapes(img, mask, img_shape=(image_size,image_size,3)):
        img.set_shape(img_shape)
        mask.set_shape([img_shape[0],img_shape[1],1])
        return img, mask
    
    ds_alb = val_ds.map(partial(process_data), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=AUTOTUNE).batch(batch)
    return ds_alb

In [ ]:
from functools import partial

batch = 4
image_size = 256
input_shape = (image_size, image_size, 3)
train_alb = augment_train_data(training_data)
val_alb = augment_val_data(validation_data)

>The above two functions tf_parse and tf_dataset are used to build the dataset pipeline.
The tf_dataset function create a tf.data pipeline which takes a list of images, masks paths and the batch size. The tf_parse function parses a single image and mask path.

# Attention based Residual Unet:
Okay! Now we are talking some fancy names xD. I have seen some NBs and found people saying that using Unet is not worth it, and I agree on that. Using MRCNN or detectron looks promising. I did not realise that earlie. So, at the last hour i though of using Unet attention based Unet,but after that I saw some code on residual attention Unet. So, I decided to give that a shot. And also there is an algorithm called `watershed` algorithm which helps to perform instance segmentation using Unet, Im thinking of doing that in future.  

## Reasons for using Attention:
<!-- ![sdfsdf](https://miro.medium.com/max/700/1*J3Rlq9K-ldxdJ5XWa4cD9g.png) -->
<p align="center">
<img width = "970" src="https://miro.medium.com/max/700/1*J3Rlq9K-ldxdJ5XWa4cD9g.png">
</p>


We all know about the Unet model, its a encoder and decoder based architecture with multi-stage cascaded convolutional neural networks. It also takes feature representation from encoder/downsampling path into consideration by concating the it the decoder/upsampling path. And after that it uses a dense layer for prediction. But, if you think carefully, it will come to your mind that the feature vectors from the starting of the downsampling path is not robust, and using that with the same level upsampling does not give much improvements. So, to address this issue we bring `attention` into the picture. `Attention` uses the `same level` downsampling feature map and also the `"feature map from one level below"` and pass that through an attention gate, which helps to extract better feature map for concating with the cooresponding upsampling level.It also helps to focus on the important part of the image that is relevant. That drags down the Time per epoch. Check out the below gradcam visualization,

<br>
<p align="center">
<img width = "970" src="https://www.researchgate.net/publication/330877339/figure/fig6/AS:863282093096962@1582833898320/The-figure-shows-the-attention-coefficients-a-l-s-2-a-l-s-3-across-different.jpg">
</p>


Now, the topic boiles down to how does `attention gates` work? Lets, Check out the below image, 

<p align="center">
<img width = "970" src="https://miro.medium.com/max/700/1*kEuDd2mCqTZkyNmkTGdg9Q.png">
</p>

> In the W_g, we have strib of 1 and for the W_x there is stribe of 2. But for both of them the number of filters are same. W_g and W_x helps to match the size of g and x. As g is coming from one level lower than the current one, thats why it has smaller shape. Now, that we have same size for both, we can go ahead and add them. Because of the addition aligned weights gets larger but the unaligned weight gets even smaller. After that we send that through `relu` activation which will make all the weights >=0. After that we pass that through a Conv block of stribe = 1 and filters = 1. Then we add a `sigmoid` layer which helps to convert the values in between the limit of 0 to 1. Then we resample it to the size of x. and multiply the x and the resampled result. This becomes the final feature map which gets concatinated with the current upsample later.



## Reasons for using Residual/skip connections:
Well when we talk about Deep Learning, the models that comes to the mind is BERT, transformer , YOLO, MRCNN etc. All these models are big in size where duzons of layers stacked together. But some times making a model too big comes with a price. Which is nothing but the `vanishing gradient problem`. It happens when the model is too big, and at the time of backpropagation the updating of the model weights becomes smaller. For dealing with this problem ResidualNet was introduced. The main point of residualnet was the skip connections. Even if the weights get smaller because we have skip connection it can't reduce the feature. Check out the below image,

<p align="center">
<img width = "500" src="https://i.imgur.com/xBk3ZGT.png">
</p>

> Think of the (a) image a conv block for Unet which is consist of two `conv+relu` layer. (b) image is same too, just with a skip connection. If the Unet model is having only the Skip connections then the model would look like this,

<br>
<br>
<p align="center">
<img width = "970" src="https://www.researchgate.net/profile/Lei-Xiang/publication/327748708/figure/fig2/AS:698665882619906@1543586335266/Illustration-of-the-proposed-Res-Unet-architecture-as-the-generator.png">
</p>


In [ ]:
def conv_block(x, filter_size, size, dropout, batch_norm=False):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv


def repeat_elem(tensor, rep):

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)


def res_conv_block(x, filter_size, size, dropout, batch_norm=False):
    

    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation('relu')(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    #conv = layers.Activation('relu')(conv)    #Activation before addition with shortcut
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    shortcut = layers.Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)

    res_path = layers.add([shortcut, conv])
    res_path = layers.Activation('relu')(res_path)    #Activation after addition with shortcut (Original residual block)
    return res_path

def gating_signal(input, out_size, batch_norm=False):
    
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn



def Attention_ResUNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    '''
    attention  baed Rsidual UNet
    
    '''
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    # input data
    # dimension of the image depth
    inputs = layers.Input(input_shape, dtype=tf.float32)
    axis = 3

    # Downsampling layers
    # DownRes 1, double residual convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=axis)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=axis)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=axis)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=axis)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=axis)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="AttentionResUNet")
    return model



# Model Training:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [ ]:
model_output = "./model_unet.h5"
# "save the Keras model or model weights at some frequency"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_output,
    save_best_only=True,
    save_weights_only=True,
)

In [ ]:
# Initialize wandb with your project name
run = wandb.init(project='kaggle sartorius',entity="somusan",
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 1e-4,
                     "epochs": 60,
                     "batch_size": 4,
                     "loss_function": "bce_dice_loss",
                     "architecture": "attention based residual Unet",
                     "dataset": "sartorius:brain cell i/s"
                 })
config = wandb.config  # We'll use this to configure our experiment


config={ 
     "learning_rate": 1e-4,
     "epochs": 60,
     "batch_size": 4,
     "loss_function": "bce_dice_loss",
     "architecture": "attention based residual Unet",
     "dataset": "sartorius:brain cell i/s"
}

In [ ]:
import segmentation_models as sm
IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 3
num_labels = 1
input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
batch = config["batch_size"]


train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1
    


att_res_unet_model = Attention_ResUNet(input_shape,dropout_rate=0.3)

opt = tf.keras.optimizers.Adam(config["learning_rate"])
# metrics = ["acc", iou]
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
att_res_unet_model.compile(loss=bce_dice_loss, optimizer=opt, metrics=metrics)


# att_res_unet_model.compile(optimizer=Adam(lr = 1e-3), loss='binary_crossentropy', 
#               metrics=['accuracy', jacard_coef])

print(att_res_unet_model.summary())

In [ ]:
att_res_unet_history = att_res_unet_model.fit(train_alb, 
                    verbose=1,
                    batch_size = batch,
                    validation_data=val_alb, 
                    steps_per_epoch=train_steps,
                    validation_steps=valid_steps,
                    shuffle=False,
                    epochs=config["epochs"],# 
                    callbacks=[model_checkpoint,WandbCallback()]) # WandbCallback()

In [ ]:
att_res_unet_model.save('./attn_res_unet_60_tta_aug_1e4_metrics_change.h5')

In [ ]:
wandb.save("./attn_res_unet_60_tta_aug_1e4_metrics_change.h5")

# Model performance: 

In [ ]:
plt.figure(figsize=(15,5))
loss = att_res_unet_history.history['loss']
val_loss = att_res_unet_history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# att_res_unet_history.history

In [ ]:
plt.figure(figsize=(15,5))
acc = att_res_unet_history.history['iou_score']
val_acc = att_res_unet_history.history['val_iou_score']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
acc = att_res_unet_history.history['f1-score']
val_acc = att_res_unet_history.history['val_f1-score']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'y', label='Training F1')
plt.plot(epochs, val_acc, 'r', label='Validation F1')
plt.title('Training and validation F1')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

# Inference time:
<!-- #### Loading another model trained on 100 epoch, you can also replace the the current trained one  -->

In [ ]:
# with CustomObjectScope({'iou': iou}):
#     model = Attention_ResUNet(input_shape,dropout_rate=0.3)
#     model.load_weights("./model_unet.h5")
model = tf.keras.models.load_model("./attn_res_unet_60_tta_aug_1e4_metrics_change.h5",custom_objects={"bce_dice_loss":bce_dice_loss,"iou_score": sm.metrics.IOUScore(threshold=0.5),"f1-score": sm.metrics.FScore(threshold=0.5)})

In [ ]:
# # restore the model file "model.h5" from a specific run by user "lavanyashukla"
# # in project "save_and_restore" from run "10pr4joa"
# best_model = wandb.restore('attn_res_unet_100_tta_aug_1e4.h5', run_path="somusan/kaggle sartorius/5335czpx")

# # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
# model = Attention_ResUNet(input_shape,dropout_rate=0.3)
# model.load_weights(best_model.name)

In [ ]:
def read_image(path):
    x = cv2.imread(path)
    x = cv2.resize(x, (256, 256))
#     x = np.expand_dims(x, axis=-1)
    x = x.astype(np.double)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    x = x.astype(np.double)
    return x

# Inference on some validation data:

In [ ]:

y_data = "../input/sartorious-nb-1-data-preprocessing-visualization"
ids = ['fe3e30f849f4','ffdb3cc02eef','0140b3c8f445','529f53145d75','b89f9cca5384','fe3e30f849f4']
sample_image_df = df[df['id'] == ids[4]]
x = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
x = read_image(x)
y_file = f"../input/sartorious-nb-1-data-preprocessing-visualization/masks/img/{sample_image_df['id'].iloc[0]}.png"

y = read_mask(y_file)
# print("y shape",y.shape)
y_pred = model.predict(np.expand_dims(x, axis=0))[0]>0.5
# print("y_pred shape",y_pred.shape)
h, w, _ = x.shape
white_line = np.ones((h, 10, 3)) * 255.0


# wandb.init(project='kaggle sartorius',entity="somusan")
           
           
height = 50
width = 50
fig, (ax1,ax2) = plt.subplots(nrows = 1,ncols = 2,figsize=(width,height))
ax = (ax1,ax2)


ax[0].imshow(y)
ax[0].set_xlabel('gt',fontsize=50)
ax[1].imshow(y_pred)
ax[1].set_xlabel('prediction',fontsize=50)

# wandb.log({"plot": fig})
# wandb.finish()

In [ ]:
def post_process(probability, threshold=0.5, min_size=300):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = []
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            a_prediction = np.zeros((520, 704), np.float32)
            a_prediction[p] = 1
            predictions.append(a_prediction)
    return predictions

In [ ]:
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

# print(rle_encoding(predictions[0]))

In [ ]:
sub_file = "../input/sartorius-cell-instance-segmentation/sample_submission.csv"
sub_df = pd.read_csv(sub_file)
sub_df.head()

# Basic TTA

In [ ]:
def tta_preds(image_dataset):
#Now that we know the transformations are working, let us extend to all predictions
    predictions = []
    for image in image_dataset:
        plt.figure(figsize=(20,20))
        pred_original = model.predict(np.expand_dims(image, axis=0))
        plt.subplot(1,5,1)
        plt.imshow(np.reshape(pred_original,(256,256)))
        
        pred_lr = model.predict(np.expand_dims(np.fliplr(image), axis=0))
        pred_lr = np.fliplr(pred_lr.squeeze(0))
        plt.subplot(1,5,2)
        plt.imshow(np.reshape(pred_lr,(256,256)))

        
        pred_ud = model.predict(np.expand_dims(np.flipud(image), axis=0))
        pred_ud = np.flipud(pred_ud.squeeze(0))
        plt.subplot(1,5,3)
        plt.imshow(np.reshape(pred_ud,(256,256)))
        
        
        
        pred_lr_ud = model.predict(np.expand_dims(np.fliplr(np.flipud(image)), axis=0))
        pred_lr_ud = np.fliplr(np.flipud(pred_lr_ud).squeeze(0))
        plt.subplot(1,5,4)
        plt.imshow(np.reshape(pred_lr_ud,(256,256)))

        
        
        preds = (pred_original + pred_lr + pred_ud + pred_lr_ud) / 4
#         plt.subplot(1,5,4)
#         plt.imshow(np.reshape(preds,(256,256)))
        
#         break
        predictions.append(preds)


    predictions = np.array(predictions)

    threshold = 0.5
    predictions_th = predictions > threshold
    
    return predictions_th

test_dir = "../input/sartorius-cell-instance-segmentation/test"
ids = sub_df["id"].tolist()

sub_list = []
sub_img = []
for i in ids:
    file_path = test_dir + "/" + i + ".png"
    x = read_image(file_path)
    sub_img.append(x)

sub_preds = tta_preds(sub_img)


for ide,j in enumerate(sub_preds):
#     print(j.astype(np.float32).shape)
#     break
    y_pred = np.reshape(j.astype(np.uint16),(256,256))
#     print(j.astype(np.float32).shape)

    y_pred = cv2.resize(y_pred,(704,520),interpolation = cv2.INTER_AREA)
#     print(y_pred.shape)
    predictions = post_process(y_pred)
    for k in predictions:
        sub_list.append((ids[ide],rle_encoding(k)))

In [ ]:
sub = pd.DataFrame(sub_list,columns=['id','predicted'])
sub.head()

In [ ]:
sub.to_csv("submission.csv",index=False)

In [ ]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()

    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    if len(shape)==3:
        img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    else:
        img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for start, end in zip(starts, ends):
        img[start : end] = color

    return img.reshape(shape)



def build_masks(df_train, image_id, input_shape):
    height, width = input_shape
    labels = df_train[df_train["id"] == image_id]["predicted"].tolist()
    mask = np.zeros((height, width))
    for label in labels:
        mask += rle_decode(label, shape=(height, width))
#         plt.imshow(mask)
#         break
    mask = mask.clip(0, 1)
    return mask

In [ ]:
sub["id"].value_counts()

In [ ]:
sample_filename = '7ae19de7bc2a'
sample_path = f"../input/sartorius-cell-instance-segmentation/test/{sample_filename}.png"
sample_img = plt.imread(sample_path)
# sample_rles = sample_image_df['annotation'].values

COLOR_MAP = 'seismic'
sample_masks1=build_masks(sub,sample_filename,input_shape=(520, 704))
# sample_masks2=build_masks(sample_filename,input_shape=(520, 704), colors=True)

fig, axs = plt.subplots(1,2, figsize=(18, 18))

axs[0].imshow(sample_img,cmap='seismic')
axs[0].set_title('original image', fontsize=16)


axs[1].imshow(sample_masks1,cmap='gray')
axs[1].set_title('mask image', fontsize=16)

plt.imshow(sample_masks1)

# Checking overlapping:

In [ ]:
def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

def combine_masks(masks, mask_threshold):
    """
    combine masks into one image
    """
    maskimg = np.zeros((HEIGHT, WIDTH))
    
    for m, mask in enumerate(masks,1):
        maskimg[mask>mask_threshold] = m
    return maskimg


def tta_preds(image_dataset):
#Now that we know the transformations are working, let us extend to all predictions
    predictions = []
    for image in image_dataset:
#         plt.figure(figsize=(20,20))
        pred_original = model.predict(np.expand_dims(image, axis=0))
#         plt.subplot(1,5,1)
#         plt.imshow(np.reshape(pred_original,(256,256)))
        
        pred_lr = model.predict(np.expand_dims(np.fliplr(image), axis=0))
        pred_lr = np.fliplr(pred_lr.squeeze(0))
#         plt.subplot(1,5,2)
#         plt.imshow(np.reshape(pred_lr,(256,256)))

        
        pred_ud = model.predict(np.expand_dims(np.flipud(image), axis=0))
        pred_ud = np.flipud(pred_ud.squeeze(0))
#         plt.subplot(1,5,3)
#         plt.imshow(np.reshape(pred_ud,(256,256)))
        
        
        
        pred_lr_ud = model.predict(np.expand_dims(np.fliplr(np.flipud(image)), axis=0))
        pred_lr_ud = np.fliplr(np.flipud(pred_lr_ud).squeeze(0))
#         plt.subplot(1,5,4)
#         plt.imshow(np.reshape(pred_lr_ud,(256,256)))

        
        
        preds = (pred_original + pred_lr + pred_ud + pred_lr_ud) / 4
#         plt.subplot(1,5,4)
#         plt.imshow(np.reshape(preds,(256,256)))
        
#         break
        predictions.append(preds)


    predictions = np.array(predictions)

    threshold = 0.5
    predictions_th = predictions > threshold
    
    return predictions_th

test_dir = "../input/sartorius-cell-instance-segmentation/test"
ids = sub_df["id"].tolist()

sub_list = []
sub_img = []
for i in ids:
    file_path = test_dir + "/" + i + ".png"
    x = read_image(file_path)
    sub_img.append(x)

sub_preds = tta_preds(sub_img)


for ide,j in enumerate(sub_preds):
#     print(j.astype(np.float32).shape)
#     break
    y_pred = np.reshape(j.astype(np.uint16),(256,256))
#     print(j.astype(np.float32).shape)

    y_pred = cv2.resize(y_pred,(704,520),interpolation = cv2.INTER_AREA)
#     print(y_pred.shape)
    predictions = post_process(y_pred)
#     for k in predictions:
#         sub_list.append((ids[ide],rle_encoding(k)))
    mask_1 = []

    for i in predictions:
        b_mask = remove_overlapping_pixels(i,mask_1)
        mask_1.append(b_mask)
        sub_list.append((ids[ide],rle_encoding(b_mask)))


        
sub_df_man = pd.DataFrame(sub_list,columns=['id','predicted'])
sub_df_man.head()

#### Seems like the same as previous df

# Credits:
1. https://github.com/MoleImg/Attention_UNet
2. https://www.kaggle.com/aramos/sartorius-competition-training-keras-unet
3. https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-training